In [2]:
import requests
import pandas as pd
import datetime as dt
import numpy as np

In [3]:
matches = pd.read_csv('matches.csv')
pd.set_option('display.max_columns', None)

In [4]:
#Sends request to get past matches

def pastMatchesRequest(date_from, date_to, competition_id, page):
    url = "https://live-score-api.p.rapidapi.com/scores/history.json"

    headers = {
        'x-rapidapi-key': "ce0be8d538msh30ecb1b4c0fca98p105373jsn1e40f1d8d1ec",
        'x-rapidapi-host': "live-score-api.p.rapidapi.com",
        }

    params = {
        'key': "MC8uSjXafFUAGJqM",
        'secret': "xRuVIJwAPNVMqsYc46HXqF7Fk2f1MGNj",
        'from': date_from,
        'to': date_to,
        'competition_id': competition_id,
        'page': page
    }

    return requests.request("GET", url, params = params,headers=headers).json()

In [5]:
#Sends request to get stats from a specific match

def matchStatsRequest(match_id):
    url = "https://live-score-api.p.rapidapi.com/matches/stats.json"

    headers = {
        'x-rapidapi-key': "ce0be8d538msh30ecb1b4c0fca98p105373jsn1e40f1d8d1ec",
        'x-rapidapi-host': "live-score-api.p.rapidapi.com",
        }

    params = {
        'key': "MC8uSjXafFUAGJqM",
        'secret': "xRuVIJwAPNVMqsYc46HXqF7Fk2f1MGNj",
        'match_id': match_id
    }

    return requests.request("GET", url, params = params,headers=headers).json()

In [119]:
#Stores information from 'Past matches' on a list

def addMatchInfo(response, continent, country, league_name, type, tier):
    for info in response['data']['match']:
        if(info['ht_score'] == ''):
            info['ht_score'] = '0 - 0'
            
        if(info['scheduled'] is None or info['scheduled'] == ""):
            matches_list.append([
                info['id'],
                info['home_id'],
                info['home_name'],
                info['away_id'],
                info['away_name'],
                info['score'].split(' -')[0],
                info['score'].split('- ')[1],
                info['ht_score'].split(' -')[0],
                info['ht_score'].split('- ')[1],
                info['location'],
                'City',
                country,
                continent,
                league_name,
                type,
                tier,
                np.nan,
                np.nan,
                info['date'].split('-')[2],
                dt.date(int(info['date'].split('-')[0]),int(info['date'].split('-')[1]),int(info['date'].split('-')[2])).strftime("%A"),
                info['date'].split('-')[1],
                info['date'].split('-')[0],
                'F'
            ])
        else:
            matches_list.append([
                info['id'],
                info['home_id'],
                info['home_name'],
                info['away_id'],
                info['away_name'],
                info['score'].split(' -')[0],
                info['score'].split('- ')[1],
                info['ht_score'].split(' -')[0],
                info['ht_score'].split('- ')[1],
                info['location'],
                'City',
                country,
                continent,
                league_name,
                type,
                tier,
                info['scheduled'].split(':')[0],
                info['scheduled'].split(':')[1],
                info['date'].split('-')[2],
                dt.date(int(info['date'].split('-')[0]),int(info['date'].split('-')[1]),int(info['date'].split('-')[2])).strftime("%A"),
                info['date'].split('-')[1],
                info['date'].split('-')[0],
                'F'
            ])

In [7]:
#Updates df with 'Match stats' information

def addMatchStats(rowIndex, matchId):
    response = matchStatsRequest(matchId)
    if(response['success'] == False):
        return False
    if(response['data'] != []):
        matches.at[rowIndex,'home_yc'] = response['data']['yellow_cards'].split(':')[0]
        matches.at[rowIndex,'away_yc'] = response['data']['yellow_cards'].split(':')[1]
        matches.at[rowIndex,'home_rc'] = response['data']['red_cards'].split(':')[0]
        matches.at[rowIndex,'away_rc'] = response['data']['red_cards'].split(':')[1]
        if(response['data']['possesion'] is None):
            matches.at[rowIndex,'home_possession'] = np.nan
            matches.at[rowIndex,'away_possession'] = np.nan
        else:
            matches.at[rowIndex,'home_possession'] = response['data']['possesion'].split(':')[0]
            matches.at[rowIndex,'away_possession'] = response['data']['possesion'].split(':')[1]
            
        if(response['data']['attempts_on_goal'] is None):
            matches.at[rowIndex,'home_shots'] = np.nan
            matches.at[rowIndex,'away_shots'] = np.nan 
        else:
            matches.at[rowIndex,'home_shots'] = response['data']['attempts_on_goal'].split(':')[0] 
            matches.at[rowIndex,'away_shots'] = response['data']['attempts_on_goal'].split(':')[1] 
            
        if(response['data']['shots_on_target'] is not None):
            matches.at[rowIndex,'home_shots_on_target'] = response['data']['shots_on_target'].split(':')[0]
            matches.at[rowIndex,'away_shots_on_target'] = response['data']['shots_on_target'].split(':')[1]
        elif(response['data']['attempts_on_goal'] is not None and response['data']['shots_off_target'] is not None):
            matches.at[rowIndex,'home_shots_on_target'] = int(response['data']['attempts_on_goal'].split(':')[0]) - int(response['data']['shots_off_target'].split(':')[0])
            matches.at[rowIndex,'away_shots_on_target'] = int(response['data']['attempts_on_goal'].split(':')[1]) - int(response['data']['shots_off_target'].split(':')[1])
        else:
            matches.at[rowIndex,'home_shots_on_target'] = np.nan
            matches.at[rowIndex,'away_shots_on_target'] = np.nan 
       
        if(response['data']['fauls'] is None):
            matches.at[rowIndex,'home_fouls'] = np.nan
            matches.at[rowIndex,'away_fouls'] = np.nan
        else:
            matches.at[rowIndex,'home_fouls'] = response['data']['fauls'].split(':')[0]
            matches.at[rowIndex,'away_fouls'] = response['data']['fauls'].split(':')[1]
        
        if(response['data']['attacks'] is None):
            matches.at[rowIndex,'home_attacks'] = np.nan
            matches.at[rowIndex,'away_attacks'] = np.nan
        else:
            matches.at[rowIndex,'home_attacks'] = response['data']['attacks'].split(':')[0]
            matches.at[rowIndex,'away_attacks'] = response['data']['attacks'].split(':')[1]
    matches.at[rowIndex,'got_stats'] = 'T'
    return True

In [120]:
#Gets past matches and stores it on a df

def getMatches(date_from, date_to, competition_id, continent, country, league_name, type, tier):
    page = 1
    
    response = pastMatchesRequest(date_from, date_to, competition_id, page)
    addMatchInfo(response, continent, country, league_name, type, tier)
    
    total_pages = response['data']['total_pages']
    page += 1
    
    while(page <= total_pages):
        response = pastMatchesRequest(date_from, date_to, competition_id, page)
        addMatchInfo(response, continent, country, league_name, type, tier)
        page += 1
    
    return pd.DataFrame(matches_list, columns =['match_id','home_id','home_name','away_id','away_name','home_ft_goals','away_ft_goals','home_ht_goals','away_ht_goals','stadium_name','city','country','continent','competition_name','type','tier','hour','minute','day','dayofweek','month','year','got_stats'])

In [9]:
#Gets match stats from top to bottom until an error is found

def getMatchStats():
    success = True
    
    while(success):
        rowIndex = matches[matches['got_stats'] == 'F'].head(2).tail(1).index
        matchId = matches[matches['got_stats'] == 'F'].head(2).tail(1)['match_id']
        success = addMatchStats(rowIndex, matchId)

# Competition Ids

1 - Bundesliga (Both)

93 - 2.Bundesliga (Both)

167 - DFB Pokal (Last)

2 - Premier League (Both)

77 - Championship (Both)

150 - EFL Cup (Last)

152 - FA Cup (Last)

3 - La Liga (Both)

79 - Spanish Segunda Division (Both)

334 - Copa Del Rey (Last)

4 - Serie A (Both)

87 - Serie B (Both)

179 - Coppa Italia (Last)

5 - Ligue 1 (Both)

162 - Coupe de France (Last)

163 - Coupe Ligue (Last)

6 - Turkish Super Lig (Both)

347 - Fortis Turkish Cup (Last) 

7 - Russian Premier League (Both)

311 - Kubok (Last) 

8 - Primeira Liga (Both)

92 - Segunda Liga (Both)

212 - Taça de Portugal (Last)

196 - Eredivisie

244 - Champions League (Both)
 
245 - Europa League (Both)

24 - Brasileirão (Both)

96 - Primera Division (Both)

329 - Copa Libertadores (Both)

76 - MLS (Both)

45 - Liga MX (Both)

26 - Chinese Super League (Both)

28 - J-League (Both)

42 - Tunisian Ligue 1 (Both)

In [155]:
matches_list = []
df = getMatches('2020-09-01','2020-12-21', 196, 'Europe', 'Netherlands', 'Eredivisie', 'League', 1)
df

,match_id,home_id,home_name,away_id,away_name,home_ft_goals,away_ft_goals,home_ht_goals,away_ht_goals,stadium_name,city,country,continent,competition_name,type,tier,hour,minute,day,dayofweek,month,year,got_stats
0,208862,583,SC Heerenveen,587,Willem II,2,0,1,0,Abe Lenstra Stadion,City,Netherlands,Europe,Eredivisie,League,1,16,45,12,Saturday,09,2020,F
1,208906,1149,PEC Zwolle,141,Feyenoord,0,2,0,1,MAC³PARK Stadion,City,Netherlands,Europe,Eredivisie,League,1,18,00,12,Saturday,09,2020,F
2,208932,582,FC Twente,339,Fortuna Sittard,2,0,1,0,De Grolsch Veste,City,Netherlands,Europe,Eredivisie,League,1,19,00,12,Saturday,09,2020,F
3,209035,346,FC Emmen,584,VVV-Venlo,3,5,1,0,De Oude Meerdijk,City,Netherlands,Europe,Eredivisie,League,1,10,15,13,Sunday,09,2020,F
4,209066,1148,Heracles,1258,ADO Den Haag,2,0,0,0,Polman Stadion,City,Netherlands,Europe,Eredivisie,League,1,12,30,13,Sunday,09,2020,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,219401,1355,FC Utrecht,586,Ajax,0,3,0,0,Stadion Galgenwaard,City,Netherlands,Europe,Eredivisie,League,1,11,15,08,Sunday,11,2020,F
67,219464,141,Feyenoord,1147,FC Groningen,2,0,0,0,Stadion Feijenoord,City,Netherlands,Europe,Eredivisie,League,1,13,30,08,Sunday,11,2020,F
68,219465,175,Vitesse,346,FC Emmen,3,1,3,0,GelreDome,City,Netherlands,Europe,Eredivisie,League,1,13,30,08,Sunday,11,2020,F
69,219532,583,SC Heerenveen,802,AZ Alkmaar,0,3,0,2,Abe Lenstra Stadion,City,Netherlands,Europe,Eredivisie,League,1,15,45,08,Sunday,11,2020,F


In [156]:
matches = matches.append(df, ignore_index = True)
matches.to_csv('matches.csv', index=False);

In [157]:
getMatchStats()
matches[matches['got_stats'] == 'F']
#matches.to_csv('matches.csv', index=False);

,match_id,home_id,home_name,away_id,away_name,home_ft_goals,away_ft_goals,home_ht_goals,away_ht_goals,stadium_name,city,country,continent,competition_name,type,tier,hour,minute,day,dayofweek,month,year,got_stats,home_yc,away_yc,home_rc,away_rc,home_possession,away_possession,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_attacks,away_attacks
3211,209873,46,Bayern Munich,45,Schalke 04,8,0,3,0,Allianz Arena,City,Bundesliga,Europe,Germany,League,1,18,30,18,Friday,09,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3253,216849,52,Wolfsburg,289,Arminia Bielefeld,2,1,2,0,VOLKSWAGEN ARENA,City,Bundesliga,Europe,Germany,League,1,14,30,25,Sunday,10,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3254,216922,56,Werder Bremen,42,Hoffenheim,1,1,1,1,Weserstadion,City,Bundesliga,Europe,Germany,League,1,17,00,25,Sunday,10,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3255,217070,57,Bayer Leverkusen,48,Augsburg,3,1,1,0,BayArena,City,Bundesliga,Europe,Germany,League,1,19,30,26,Monday,10,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3256,217542,45,Schalke 04,54,VfB Stuttgart,1,1,1,0,VELTINS-Arena,City,Bundesliga,Europe,Germany,League,1,19,30,30,Friday,10,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10017,219401,1355,FC Utrecht,586,Ajax,0,3,0,0,Stadion Galgenwaard,City,Netherlands,Europe,Eredivisie,League,1,11,15,08,Sunday,11,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10018,219464,141,Feyenoord,1147,FC Groningen,2,0,0,0,Stadion Feijenoord,City,Netherlands,Europe,Eredivisie,League,1,13,30,08,Sunday,11,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10019,219465,175,Vitesse,346,FC Emmen,3,1,3,0,GelreDome,City,Netherlands,Europe,Eredivisie,League,1,13,30,08,Sunday,11,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10020,219532,583,SC Heerenveen,802,AZ Alkmaar,0,3,0,2,Abe Lenstra Stadion,City,Netherlands,Europe,Eredivisie,League,1,15,45,08,Sunday,11,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
pd.set_option('display.max_rows', 30)
tmp = matches.sort_values(by=['year','month', 'day'])
t = tmp[tmp['home_yc'] >= 0]
t[t['competition_name'] == 'Premier League']

,match_id,home_id,home_name,away_id,away_name,home_ft_goals,away_ft_goals,home_ht_goals,away_ht_goals,stadium_name,city,country,continent,competition_name,type,tier,hour,minute,day,dayofweek,month,year,got_stats,home_yc,away_yc,home_rc,away_rc,home_possession,away_possession,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_attacks,away_attacks
127,172252,7,Liverpool,12,Manchester City,3,1,2,0,Anfield,City,England,Europe,Premier League,League,1,16,30,10,Sunday,11,2019,T,0.0,2.0,0.0,0.0,48.0,52.0,9.0,11.0,5.0,3.0,10.0,5.0,82.0,91.0
130,173660,18,Arsenal,4,Southampton,2,2,1,1,Emirates Stadium,City,England,Europe,Premier League,League,1,15,0,23,Saturday,11,2019,T,6.0,2.0,0.0,0.0,61.0,39.0,7.0,17.0,5.0,6.0,13.0,18.0,103.0,80.0
135,173801,12,Manchester City,17,Chelsea,2,1,2,1,Etihad Stadium,City,England,Europe,Premier League,League,1,17,30,23,Saturday,11,2019,T,1.0,1.0,0.0,0.0,47.0,53.0,11.0,8.0,4.0,2.0,5.0,8.0,111.0,93.0
136,174134,281,Sheffield United,19,Manchester United,3,3,1,0,Bramall Lane,City,England,Europe,Premier League,League,1,16,30,24,Sunday,11,2019,T,1.0,2.0,0.0,0.0,41.0,59.0,9.0,9.0,8.0,5.0,11.0,10.0,90.0,104.0
137,174268,495,Aston Villa,9,Newcastle United,2,0,2,0,Villa Park,City,England,Europe,Premier League,League,1,20,0,25,Monday,11,2019,T,1.0,1.0,0.0,0.0,66.0,34.0,13.0,8.0,8.0,6.0,10.0,9.0,97.0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,201875,6,Leicester City,19,Manchester United,0,2,0,0,King Power Stadium,City,England,Europe,Premier League,League,1,15,0,26,Sunday,7,2020,T,1.0,5.0,1.0,0.0,46.0,54.0,10.0,5.0,3.0,3.0,12.0,11.0,92.0,84.0
375,201876,12,Manchester City,492,Norwich City,5,0,2,0,Etihad Stadium,City,England,Europe,Premier League,League,1,15,0,26,Sunday,7,2020,T,1.0,1.0,0.0,0.0,74.0,26.0,22.0,5.0,10.0,4.0,7.0,4.0,159.0,60.0
376,201877,9,Newcastle United,7,Liverpool,1,3,1,1,St. James' Park,City,England,Europe,Premier League,League,1,15,0,26,Sunday,7,2020,T,1.0,0.0,0.0,0.0,26.0,74.0,3.0,11.0,2.0,6.0,11.0,5.0,77.0,173.0
377,201878,4,Southampton,281,Sheffield United,3,1,0,1,St. Mary's Stadium,City,England,Europe,Premier League,League,1,15,0,26,Sunday,7,2020,T,0.0,1.0,0.0,0.0,72.0,28.0,7.0,3.0,4.0,3.0,9.0,16.0,153.0,67.0


In [13]:
matches[matches['home_yc'] >= 0]

,match_id,home_id,home_name,away_id,away_name,home_ft_goals,away_ft_goals,home_ht_goals,away_ht_goals,stadium_name,city,country,continent,competition_name,type,tier,hour,minute,day,dayofweek,month,year,got_stats,home_yc,away_yc,home_rc,away_rc,home_possession,away_possession,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_attacks,away_attacks
127,172252,7,Liverpool,12,Manchester City,3,1,2,0,Anfield,City,England,Europe,Premier League,League,1,16,30,10,Sunday,11,2019,T,0.0,2.0,0.0,0.0,48.0,52.0,9.0,11.0,5.0,3.0,10.0,5.0,82.0,91.0
130,173660,18,Arsenal,4,Southampton,2,2,1,1,Emirates Stadium,City,England,Europe,Premier League,League,1,15,0,23,Saturday,11,2019,T,6.0,2.0,0.0,0.0,61.0,39.0,7.0,17.0,5.0,6.0,13.0,18.0,103.0,80.0
135,173801,12,Manchester City,17,Chelsea,2,1,2,1,Etihad Stadium,City,England,Europe,Premier League,League,1,17,30,23,Saturday,11,2019,T,1.0,1.0,0.0,0.0,47.0,53.0,11.0,8.0,4.0,2.0,5.0,8.0,111.0,93.0
136,174134,281,Sheffield United,19,Manchester United,3,3,1,0,Bramall Lane,City,England,Europe,Premier League,League,1,16,30,24,Sunday,11,2019,T,1.0,2.0,0.0,0.0,41.0,59.0,9.0,9.0,8.0,5.0,11.0,10.0,90.0,104.0
137,174268,495,Aston Villa,9,Newcastle United,2,0,2,0,Villa Park,City,England,Europe,Premier League,League,1,20,0,25,Monday,11,2019,T,1.0,1.0,0.0,0.0,66.0,34.0,13.0,8.0,8.0,6.0,10.0,9.0,97.0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,201787,153,Braga,143,FC Porto,2,1,0,1,Estádio Municipal de Braga,City,Primeira Liga,Europe,Portugal,League,1,20,15,25,Saturday,7,2020,T,6.0,3.0,0.0,0.0,45.0,55.0,9.0,6.0,5.0,4.0,18.0,14.0,81.0,101.0
2517,201788,131,Benfica,138,Sporting CP,2,1,1,0,Estádio do Sport Lisboa e Benfica,City,Primeira Liga,Europe,Portugal,League,1,20,15,25,Saturday,7,2020,T,4.0,3.0,0.0,0.0,46.0,54.0,13.0,10.0,4.0,5.0,20.0,15.0,112.0,107.0
2518,201929,816,Moreirense,650,Tondela,1,2,0,1,Parque Desportivo Comendador Joaquim de Almeid...,City,Primeira Liga,Europe,Portugal,League,1,18,30,26,Sunday,7,2020,T,2.0,1.0,0.0,0.0,61.0,39.0,7.0,6.0,4.0,4.0,16.0,19.0,86.0,65.0
2519,201930,358,Portimonense,821,Aves,2,0,0,0,Estádio Municipal de Portimão,City,Primeira Liga,Europe,Portugal,League,1,18,30,26,Sunday,7,2020,T,2.0,3.0,0.0,0.0,57.0,43.0,8.0,7.0,6.0,2.0,19.0,21.0,157.0,97.0
